In [1]:
from src.pipeline import get_infer_pipeline
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2

__file__ = ""

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


In [2]:
class CFG:
    wandb_mode = "disabled"
    exp_name = Path(__file__).stem

    seed = 42
    epochs = 30
    img_size = 384
    n_fold = 6

    pipeline = "base"
    preprocess = "base"
    dataset = "base"
    model_name: str = "base"
    encoder_name: str = "tf_efficientnet_b0_ns"

    train_batch_size = 32
    valid_batch_size = 64
    learning_rate = 1e-4

    lossfn = "BCEWithLogitsLoss"
    sampling_factor = 20
    loss_weight = 1
    optimizer = "AdamW"
    scheduler = "CosineAnnealingLR"

    min_lr = 1e-7
    weight_decay = 1e-6

    train_transform = A.Compose(
        [
            A.Resize(img_size, img_size),
            A.RandomRotate90(p=0.5),
            A.Flip(p=0.5),
            A.Downscale(p=0.25),
            A.ShiftScaleRotate(
                shift_limit=0.1, scale_limit=0.15, rotate_limit=60, p=0.5
            ),
            A.HueSaturationValue(
                hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
            ),
            A.RandomBrightnessContrast(
                brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
            ),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0,
            ),
            ToTensorV2(),
        ],
        p=1.0,
    )

    valid_transform = A.Compose(
        [
            A.Resize(img_size, img_size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0,
            ),
            ToTensorV2(),
        ],
        p=1.0,
    )

In [3]:
CFG.weight_path = "/kaggle/weights/tmp_weight/tmp.pth"

In [4]:
pipeline = get_infer_pipeline(CFG)
pipeline(CFG)

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/tf_efficientnet_b0.ns_jft_in1k)


INFO:timm.models._hub:[timm/tf_efficientnet_b0.ns_jft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
100%|██████████| 1/1 [00:00<00:00,  5.41it/s]
